# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vaca_df = pd.read_csv("output_data/cities.csv")
vaca_df.head()

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness
0,tara,47.390047,0.688927,Clear,48.33,88,1018,3.44,0
1,klaksvik,62.226608,-6.585271,Rain,49.24,93,979,22.48,90
2,meadow lake,54.130260,-108.435059,Clouds,8.58,72,1025,4.61,100
3,punta arenas,-53.162569,-70.907822,Clouds,51.30,43,1007,19.51,89
4,avarua,-21.207474,-159.770815,Clouds,76.98,73,1018,13.80,20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
data = vaca_df.loc[:, ["latitude", "longitude"]]

dimensions = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

figure = gmaps.figure(layout=dimensions)
heatmap = gmaps.heatmap_layer(data, weights=vaca_df.humidity)
figure.add_layer(heatmap)
figure

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
mask = (vaca_df.wind_speed < 10) & (vaca_df.temperature < 80) & (vaca_df.temperature > 70) & (vaca_df.weather == "Clear")
preferred_df = vaca_df.loc[mask].reset_index(drop=True)
preferred_df

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness
0,thohoyandou,-22.967643,30.459658,Clear,74.93,54,1010,2.66,8
1,vila velha,-20.329704,-40.292017,Clear,74.95,82,1016,5.75,0
2,guhagar,17.483494,73.190034,Clear,73.49,55,1012,5.95,3
3,arlit,18.739079,7.393331,Clear,76.24,15,1014,9.86,0
4,zhaoqing,23.050166,112.460363,Clear,73.83,90,1016,1.05,10
5,dauriya,25.099725,79.398572,Clear,71.26,51,1015,4.56,2
6,natalio,-26.756470,-55.136714,Clear,78.37,41,1009,1.03,3
7,lagunillas,20.461545,-100.294437,Clear,73.85,31,1013,7.96,0
8,kaduna,10.518290,7.435986,Clear,74.98,29,1013,6.20,4
9,grand gaube,-20.008619,57.664416,Clear,71.38,71,1017,1.99,9


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = preferred_df
hotel_df.head()

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness
0,thohoyandou,-22.967643,30.459658,Clear,74.93,54,1010,2.66,8
1,vila velha,-20.329704,-40.292017,Clear,74.95,82,1016,5.75,0
2,guhagar,17.483494,73.190034,Clear,73.49,55,1012,5.95,3
3,arlit,18.739079,7.393331,Clear,76.24,15,1014,9.86,0
4,zhaoqing,23.050166,112.460363,Clear,73.83,90,1016,1.05,10


In [20]:
hotels = []

for indx, row in hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"]) > 0):
        hotel = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
    else:
        hotel = None
        address = None
        rating = np.nan
        user_ratings_total = np.nan
        
    hotel = {"Hotel Name" : hotel,
            "address": address,
            "rating": rating,
            "user_ratings_total": user_ratings_total}
    
    hotels.append(hotel)

In [21]:
df = pd.DataFrame(hotels)
df.head()

,Hotel Name,address,rating,user_ratings_total
0,2Ten Hotel and convention center,"Mphephu Street, Sibasa, Thohoyandou",4.2,1119.0
1,Hotel Prainha,"136 R. Vasco Coutinho, 136 Centro de Vila Velh...",4.0,482.0
2,"Mango Village, Guhagar","Kirtan Wadi Road, Opposite Government Rest Hou...",4.2,365.0
3,Hôtel Temet,"P9RX+27R, Arlit",3.8,84.0
4,Wyndham Grand Zhaoqing Downtown,"13 Xinghu Blvd, Road, Zhaoqing",0.0,0.0


In [22]:
hotel_full = pd.merge(hotel_df, df, left_index=True, right_index=True)
hotel_full

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness,Hotel Name,address,rating,user_ratings_total
0,thohoyandou,-22.967643,30.459658,Clear,74.93,54,1010,2.66,8,2Ten Hotel and convention center,"Mphephu Street, Sibasa, Thohoyandou",4.2,1119.0
1,vila velha,-20.329704,-40.292017,Clear,74.95,82,1016,5.75,0,Hotel Prainha,"136 R. Vasco Coutinho, 136 Centro de Vila Velh...",4.0,482.0
2,guhagar,17.483494,73.190034,Clear,73.49,55,1012,5.95,3,"Mango Village, Guhagar","Kirtan Wadi Road, Opposite Government Rest Hou...",4.2,365.0
3,arlit,18.739079,7.393331,Clear,76.24,15,1014,9.86,0,Hôtel Temet,"P9RX+27R, Arlit",3.8,84.0
4,zhaoqing,23.050166,112.460363,Clear,73.83,90,1016,1.05,10,Wyndham Grand Zhaoqing Downtown,"13 Xinghu Blvd, Road, Zhaoqing",0.0,0.0
5,dauriya,25.099725,79.398572,Clear,71.26,51,1015,4.56,2,Hotel Eva Galaxy,"ROAD, near PWD CHAWRAHA, Nowgong",4.7,111.0
6,natalio,-26.756470,-55.136714,Clear,78.37,41,1009,1.03,3,None,None,NaN,NaN
7,lagunillas,20.461545,-100.294437,Clear,73.85,31,1013,7.96,0,Finca Zerhua,"QRO 430, Sanfandila",5.0,3.0
8,kaduna,10.518290,7.435986,Clear,74.98,29,1013,6.20,4,Asaa Pyramid Hotel,"13 Lafia Road, Kaduna",3.9,1988.0
9,grand gaube,-20.008619,57.664416,Clear,71.38,71,1017,1.99,9,LUX Grand Gaube,"MU, Coastal Road, Grand Gaube",4.7,2068.0


In [31]:
# Borrowed this code cell with some minor tweaks from the course instructor. Could not figure out how to get the dynammic
# "Hotel Name" to be included in the pop-ups.

infos = []

for indx, row in hotel_full.iterrows():
    info_box_template = f"""<dl>
                        <dt>Hotel Name:</dt><dd>{'Hotel Name'}</dd>
                        <dt>Rating:</dt><dd>{row.rating}</dd>
                        <dt>Address:</dt><dd>{row.address}</dd>
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

'<dl>\n                        <dt>Hotel Name:</dt><dd>Hotel Name</dd>\n                        <dt>Rating:</dt><dd>4.2</dd>\n                        <dt>Address:</dt><dd>Mphephu Street, Sibasa, Thohoyandou</dd>\n                        <dt>Location:</dt><dd>(-22.9676429, 30.4596582)</dd>\n                        </dl>\n                        '

In [34]:
# Also Borrowed from the course instructor with some minor tweaks. 

# get data
data = vaca_df.loc[:, ["latitude", "longitude"]]
hotel_data = hotel_full.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=vaca_df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(hotel_data, info_box_content=infos)
fig.add_layer(markers)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

In [25]:
# Could not understand how to make this work as it is written without changes, as the code notes say.

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_full.iterrows()]
locations = hotel_full[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
